In [1]:
import requests
import io

from datetime import date, timedelta

url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))
c.head(3)

,date,location,new_cases,new_deaths,total_cases,total_deaths
0,2019-12-31,Afghanistan,0,0,0,0
1,2020-01-01,Afghanistan,0,0,0,0
2,2020-01-02,Afghanistan,0,0,0,0


In [2]:
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv")

In [3]:
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-04-10,France,91163.0,0.0,13934.0,0.0,0.0,0.0
0,2020-04-10,China,83100.0,0.0,3340.0,0.0,0.0,0.0
0,2020-04-10,Italy,147260.0,0.0,18738.0,0.0,0.0,0.0
0,2020-04-10,Spain,157910.0,0.0,15883.0,0.0,0.0,0.0
0,2020-04-10,United States,500160.0,0.0,18921.0,0.0,0.0,0.0
0,2020-04-10,World,1656130.0,0.0,102707.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")

In [6]:
frames = [follow_df1,follow_df2,follow_df3,follow_df4,follow_df5,follow_df6]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-04-11,France,91163.0,90676.0,13934.0,13197.0,487.0,737.0
0,2020-04-11,China,83100.0,83004.0,3340.0,3343.0,96.0,-3.0
0,2020-04-11,Italy,147260.0,147577.0,18738.0,18851.0,-317.0,-113.0
0,2020-04-11,Spain,157910.0,157022.0,15883.0,15843.0,888.0,40.0
0,2020-04-11,United States,500160.0,501560.0,18921.0,18777.0,-1400.0,144.0
0,2020-04-11,World,1656130.0,1653204.0,102707.0,102088.0,2926.0,619.0


In [7]:
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")